# FPGAで取得した2光子検出のデータから、同時検出タイミングのヒストグラムを作成

In [1]:
from pathlib import Path

from japanize_matplotlib import japanize
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm

%matplotlib inline
sns.set()
sns.set_style('whitegrid')
japanize()

In [2]:
path_here = Path().cwd()
print(path_here)

/Users/sakumadaisuke/dev/optics/tdc


In [3]:
target_file = path_here / '2025-09-20_13.12.01-GPX2_results3-rst_times.csv'

df = pd.read_csv(target_file)
display(df)

,t_stop1_ps,t_stop1_ns,ovf_stop1,refno_unwrapped_stop1,t_stop4_ps,t_stop4_ns,ovf_stop4,refno_unwrapped_stop4,STOP1,REFNO1,STOP4,REFNO4,active_stop1,active_stop4
0,0.000000e+00,0.000000e+00,0.0,605.0,0.000000e+00,0.000000e+00,0.0,2572.0,60103.0,605.0,33638.0,2572.0,True,True
1,1.799484e+08,1.799484e+05,0.0,2405.0,3.902362e+07,3.902362e+04,0.0,2962.0,8460.0,2405.0,57258.0,2962.0,True,True
2,3.040264e+08,3.040264e+05,0.0,3645.0,2.905385e+08,2.905385e+05,1.0,5477.0,86493.0,3645.0,72133.0,1381.0,True,True
3,5.525147e+08,5.525147e+05,1.0,6130.0,4.407315e+08,4.407315e+05,1.0,6979.0,74843.0,2034.0,65175.0,2883.0,True,True
4,6.606271e+08,6.606271e+05,1.0,7211.0,5.755478e+08,5.755478e+05,2.0,8327.0,87168.0,3115.0,81439.0,135.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2132,9.674182e+10,9.674182e+07,236.0,968023.0,1.905029e+11,1.905029e+08,465.0,1907601.0,81801.0,1367.0,57429.0,2961.0,True,True
2133,9.690938e+10,9.690938e+07,236.0,969699.0,1.908442e+11,1.908442e+08,466.0,1911014.0,40050.0,3043.0,61415.0,2278.0,True,True
2134,9.721980e+10,9.721980e+07,237.0,972803.0,1.908546e+11,1.908546e+08,466.0,1911118.0,55856.0,2051.0,19889.0,2382.0,True,True
2135,9.729447e+10,9.729447e+07,237.0,973550.0,1.909192e+11,1.909192e+08,466.0,1911764.0,27504.0,2798.0,30048.0,3028.0,True,True


## 一つになっているファイルをチャンネル別に分ける

In [4]:
cols_ch1 = ['t_stop1_ps', 't_stop1_ns', 'ovf_stop1', 'refno_unwrapped_stop1', 'STOP1', 'REFNO1']
cols_ch4 = ['t_stop4_ps', 't_stop4_ns', 'ovf_stop4', 'refno_unwrapped_stop4', 'STOP4', 'REFNO4']

df_ch1 = df[cols_ch1].copy()
df_ch4 = df[cols_ch4].copy()

display(df_ch1)
display(df_ch4)

,t_stop1_ps,t_stop1_ns,ovf_stop1,refno_unwrapped_stop1,STOP1,REFNO1
0,0.000000e+00,0.000000e+00,0.0,605.0,60103.0,605.0
1,1.799484e+08,1.799484e+05,0.0,2405.0,8460.0,2405.0
2,3.040264e+08,3.040264e+05,0.0,3645.0,86493.0,3645.0
3,5.525147e+08,5.525147e+05,1.0,6130.0,74843.0,2034.0
4,6.606271e+08,6.606271e+05,1.0,7211.0,87168.0,3115.0
...,...,...,...,...,...,...
2132,9.674182e+10,9.674182e+07,236.0,968023.0,81801.0,1367.0
2133,9.690938e+10,9.690938e+07,236.0,969699.0,40050.0,3043.0
2134,9.721980e+10,9.721980e+07,237.0,972803.0,55856.0,2051.0
2135,9.729447e+10,9.729447e+07,237.0,973550.0,27504.0,2798.0


,t_stop4_ps,t_stop4_ns,ovf_stop4,refno_unwrapped_stop4,STOP4,REFNO4
0,0.000000e+00,0.000000e+00,0.0,2572.0,33638.0,2572.0
1,3.902362e+07,3.902362e+04,0.0,2962.0,57258.0,2962.0
2,2.905385e+08,2.905385e+05,1.0,5477.0,72133.0,1381.0
3,4.407315e+08,4.407315e+05,1.0,6979.0,65175.0,2883.0
4,5.755478e+08,5.755478e+05,2.0,8327.0,81439.0,135.0
...,...,...,...,...,...,...
2132,1.905029e+11,1.905029e+08,465.0,1907601.0,57429.0,2961.0
2133,1.908442e+11,1.908442e+08,466.0,1911014.0,61415.0,2278.0
2134,1.908546e+11,1.908546e+08,466.0,1911118.0,19889.0,2382.0
2135,1.909192e+11,1.909192e+08,466.0,1911764.0,30048.0,3028.0


## ch1に対するch4の到達タイミングを全件数総当たりで求める

In [5]:
max_time_diff_psec = 1000000

all_time_diffs = list()
for _idx1, record_ch1 in df_ch1.iterrows():
    time_diffs = list()
    for _idx4, record_ch4 in df_ch4.iterrows():
        time_diff = record_ch4['t_stop4_ps'] - record_ch1['t_stop1_ps']
        if time_diff < 0 or time_diff > max_time_diff_psec:
            continue
        time_diffs.append(
            {
                'time_diff': time_diff,
                't_stop4_ps': record_ch4['t_stop4_ps'],
                't_stop1_ps': record_ch1['t_stop1_ps'],
            }
        )
    df_time_diff = pd.DataFrame(time_diffs)
    all_time_diffs.append(df_time_diff)
df_all_time_diff = pd.concat(all_time_diffs)
display(df_all_time_diff)

,time_diff,t_stop4_ps,t_stop1_ps
0,0.0,0.000000e+00,0.000000e+00
0,437117.0,1.747056e+09,1.746618e+09
0,501728.0,4.128788e+09,4.128287e+09
0,348882.0,8.249709e+08,8.246220e+08
0,844029.0,1.801662e+10,1.801578e+10
0,396350.0,2.398274e+10,2.398235e+10
0,730989.0,5.478236e+10,5.478163e+10
0,813119.0,5.779968e+10,5.779887e+10
0,150884.0,5.870838e+10,5.870823e+10
0,606474.0,7.895471e+10,7.895410e+10


In [6]:
len(df_ch1) * len(df_ch4)

4566769

## 任意のbinの幅で集計する

In [9]:
time_bin_psec = 10000

min_time_diff = df_all_time_diff['time_diff'].min()
max_time_diff = df_all_time_diff['time_diff'].max()

dts = np.arange(min_time_diff, max_time_diff, time_bin_psec)
print(f'最小: {min_time_diff}, 最大: {max_time_diff}, 幅: {time_bin_psec}')
print(dts)

最小: 0.0, 最大: 965493.0, 幅: 10000
[     0.  10000.  20000.  30000.  40000.  50000.  60000.  70000.  80000.
  90000. 100000. 110000. 120000. 130000. 140000. 150000. 160000. 170000.
 180000. 190000. 200000. 210000. 220000. 230000. 240000. 250000. 260000.
 270000. 280000. 290000. 300000. 310000. 320000. 330000. 340000. 350000.
 360000. 370000. 380000. 390000. 400000. 410000. 420000. 430000. 440000.
 450000. 460000. 470000. 480000. 490000. 500000. 510000. 520000. 530000.
 540000. 550000. 560000. 570000. 580000. 590000. 600000. 610000. 620000.
 630000. 640000. 650000. 660000. 670000. 680000. 690000. 700000. 710000.
 720000. 730000. 740000. 750000. 760000. 770000. 780000. 790000. 800000.
 810000. 820000. 830000. 840000. 850000. 860000. 870000. 880000. 890000.
 900000. 910000. 920000. 930000. 940000. 950000. 960000.]


In [13]:
counter = {int(dt): 0 for dt in dts}

for _idx, record in tqdm(df_all_time_diff.iterrows(), total=len(df_all_time_diff), leave=True):
    for dt in reversed(counter.keys()):
        if record['time_diff'] >= dt:
            counter[dt] += 1
            break
sr_coincedence = pd.Series(counter)
display(counter)

  0%|          | 0/27 [00:00<?, ?it/s]

{0: 1,
 10000: 0,
 20000: 0,
 30000: 0,
 40000: 0,
 50000: 0,
 60000: 0,
 70000: 0,
 80000: 0,
 90000: 0,
 100000: 0,
 110000: 0,
 120000: 0,
 130000: 0,
 140000: 0,
 150000: 1,
 160000: 0,
 170000: 1,
 180000: 0,
 190000: 0,
 200000: 1,
 210000: 1,
 220000: 1,
 230000: 0,
 240000: 0,
 250000: 1,
 260000: 0,
 270000: 0,
 280000: 0,
 290000: 1,
 300000: 0,
 310000: 0,
 320000: 0,
 330000: 0,
 340000: 1,
 350000: 0,
 360000: 0,
 370000: 1,
 380000: 0,
 390000: 1,
 400000: 0,
 410000: 0,
 420000: 0,
 430000: 1,
 440000: 0,
 450000: 0,
 460000: 0,
 470000: 0,
 480000: 0,
 490000: 0,
 500000: 1,
 510000: 0,
 520000: 0,
 530000: 0,
 540000: 0,
 550000: 0,
 560000: 0,
 570000: 0,
 580000: 0,
 590000: 0,
 600000: 1,
 610000: 1,
 620000: 0,
 630000: 0,
 640000: 0,
 650000: 1,
 660000: 0,
 670000: 0,
 680000: 0,
 690000: 0,
 700000: 0,
 710000: 1,
 720000: 0,
 730000: 1,
 740000: 0,
 750000: 0,
 760000: 0,
 770000: 0,
 780000: 0,
 790000: 0,
 800000: 0,
 810000: 1,
 820000: 0,
 830000: 0,
 84000

## 可視化

- 横軸: 時間
- 縦軸: カウント